In [ ]:
import mne
edf_path = "../data/physionet.org/files/chbmit/1.0.0/chb01/chb01_01.edf"
raw = mne.io.read_raw_edf(edf_path, preload=True)
raw.info

In [ ]:
# Apply band-pass filter (0.5–40 Hz)
raw.filter(0.5, 40., fir_design='firwin')

# Downsample to 128 Hz
raw.resample(128)

# Plot a short segment (first 10 seconds, 10 channels)
raw.plot(duration=10, n_channels=10)

In [ ]:
import numpy as np

# Extract the data and channel names
data, times = raw.get_data(return_times=True)
print("EEG shape:", data.shape)
print("Time duration (s):", times[-1])

In [ ]:
def create_windows(data, sfreq=128, window_size=30, stride=10):
    samples_per_window = int(window_size * sfreq)
    step = int(stride * sfreq)
    windows = []
    for start in range(0, data.shape[1] - samples_per_window, step):
        end = start + samples_per_window
        windows.append(data[:, start:end])
    return np.array(windows)

# Create windows (30s window, 10s stride)
windows = create_windows(data, sfreq=128, window_size=30, stride=10)
print("Windows shape:", windows.shape)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 4))
plt.plot(windows[0][0][:1000])  # first channel, 1000 samples (~8 seconds)
plt.title("EEG signal (first window, first channel)")
plt.xlabel("Samples")
plt.ylabel("Amplitude (µV)")
plt.show()

In [ ]:
import re
import numpy as np
import pandas as pd

# Path to your summary file (update this with your actual path)
summary_file = "../data/physionet.org/files/chbmit/1.0.0/chb01/chb01-summary.txt"

# Read summary file
with open(summary_file, "r") as f:
    summary_text = f.read()

# Extract seizure intervals for each record using regex
pattern = r"File Name: (\S+\.edf).*?Seizure Start Time: (\d+) seconds.*?Seizure End Time: (\d+) seconds"
matches = re.findall(pattern, summary_text, re.DOTALL)

# Store seizure intervals in a dictionary
seizure_dict = {}
for file_name, start, end in matches:
    if file_name not in seizure_dict:
        seizure_dict[file_name] = []
    seizure_dict[file_name].append((int(start), int(end)))

print("Extracted seizure intervals:")
for k, v in seizure_dict.items():
    print(f"{k}: {v}")

# Example: labeling for one EDF file
edf_name = "chb01_03.edf"  # change this to match your loaded file
seizure_intervals = seizure_dict.get(edf_name, [])

# Label creation
window_size = 3840 / 256  # 15 seconds
window_starts = np.arange(0, len(windows)) * window_size
labels = np.zeros(len(windows))

for start, end in seizure_intervals:
    for i, win_start in enumerate(window_starts):
        win_end = win_start + window_size
        if (win_start < end) and (win_end > start):
            labels[i] = 1

labels_df = pd.DataFrame({
    "window_index": np.arange(len(windows)),
    "start_time_sec": window_starts,
    "label": labels
})

print(labels_df.head(10))
print(f"Seizure windows: {np.sum(labels==1)}, Non-seizure: {np.sum(labels==0)}")
